In [1]:
import numpy as np
import pandas as pd
import dill as dl
import matplotlib.pyplot as plt
import sys
import pickle
import itertools
import re
import os
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as sm_f

In [2]:
with open("D:/DZ/Course_6/Diploma/Data/two_time_points.csv", "rb") as inp:
    two_time_points = pickle.load(inp)

In [3]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [4]:
with open("D:/DZ/Course_6/Diploma/Data/mun_data_50.csv", "rb") as inp:
    data_for_models = pd.read_csv(inp, index_col=0)

In [5]:
with open("D:/DZ/Course_6/Diploma/Data/full_ind_data.csv", "rb") as inp:
    full_ind_data = pickle.load(inp)

In [6]:
#with open("D:/DZ/Course_6/Diploma/Data/mun_data_25.csv", "rb") as inp:
#    mun_data = pd.read_csv(inp, index_col=0)

In [7]:
pd.set_option("max_columns", None)

In [8]:
two_time_points

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_period
0,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0
1,Владивостокский,Приморский край,1,1,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,1
2,Хасанский район,Приморский край,0,1,2016,NaN,0.0,NaN,4.20,32139.0,NaN,0.023735,0.011735,NaN,0
3,Хасанский район,Приморский край,0,1,2021,NaN,NaN,NaN,6.20,30318.0,NaN,0.020031,0.004033,0.002294,1
4,Артемовский,Приморский край,1,1,2016,NaN,0.0,NaN,2.62,114873.0,NaN,0.009542,0.009249,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Мошенской район,Новгородская область,0,0,2021,45.00,NaN,NaN,3.32,5797.0,NaN,NaN,0.007950,NaN,1
304,Пестовский район,Новгородская область,0,1,2016,88.77,0.0,NaN,0.63,20934.0,NaN,NaN,0.003777,NaN,0
305,Пестовский район,Новгородская область,0,1,2021,75.10,NaN,NaN,0.82,19579.0,NaN,NaN,0.009602,0.014286,1
306,Хвойнинский район,Новгородская область,0,1,2016,27.61,10.0,NaN,24.65,14950.0,NaN,NaN,0.003137,0.012089,0


In [9]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [10]:
temp_fin['values'].min()

-46465.5756015087

In [11]:
full_ind_data.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201


In [12]:
full_ind_data = pd.merge(full_ind_data, data_for_models, on=["raion", "region"], how="left").drop(columns=["treatment_1_x", "treatment_2_x"]).rename(columns={"treatment_1_y": "treatment_1", "treatment_2_y": "treatment_2"}).reset_index(drop=True)

In [13]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

# Goods

In [14]:
full_ind_data.year.max()

2021

In [15]:
for i in range(-8,4):
    if i < 0:
        string = f"minus_{abs(i)}"
    else:
        string = f'plus_{i}'
    full_ind_data[string] = 0
    for region in mapping.keys():
        full_ind_data.loc[(full_ind_data.region == region) & (full_ind_data.year == mapping[region]+5+i) & (full_ind_data.treatment_1 == 1), string] = 1

In [16]:
full_ind_data.head(10)

,raion,region,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Владивостокский,Приморский край,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Владивостокский,Приморский край,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Владивостокский,Приморский край,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Владивостокский,Приморский край,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Владивостокский,Приморский край,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,Владивостокский,Приморский край,2018,NaN,0.0,34023.34,4.21,633102.0,NaN,0.016002,0.005096,0.002653,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,Владивостокский,Приморский край,2019,NaN,0.0,67570.22,4.48,633144.0,NaN,0.028937,0.013167,0.002075,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,Владивостокский,Приморский край,2020,NaN,NaN,106191.72,4.80,634665.0,NaN,0.025499,0.007071,0.001982,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,Владивостокский,Приморский край,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,Хасанский район,Приморский край,2013,NaN,6.3,116999.00,4.30,34112.0,NaN,0.028997,0.009722,0.008197,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
treat_1_roads = full_ind_data[["raion", "region", "bad_roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", 
                 data=treat_1_roads).fit()
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     38.98
Date:                Fri, 12 May 2023   Prob (F-statistic):          4.16e-279
Time:                        23:03:25   Log-Likelihood:                -3514.4
No. Observations:                 955   AIC:                             7301.
Df Residuals:                     819   BIC:                             7962.
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           11.6875      3.847      3.038      0.002       4.137      19.239
C(raion)[T.Антроповский район]                                      75.1492      5.035     14.925      0.000      65.266      85.032
C(raion)[T.Аяно-Майский район]                                      88.3125     11.050      7.992      0.000      66.623     110.002
C(raion)[T.Батецкий район]                                          51.7447      5.035     10.277      0.000      41.861      61.628
C(raion)[T.Бикинский район]                                         21.2425      5.179      4.101      0.000      11.076      31.409
C(raion)[T.Боровичский район]                                       28.5875      6.072      4.708      0.000      16.669      40.506
C(raion)[T.Буйский район]                                           30.2658      5.035      6.011      0.000      20.383      40.149
C(raion)[T.Валдайский район]                                        27.6814      5.035      5.498      0.000      17.798      37.565
C(raion)[T.Ванинский район]                                         13.0312      5.179      2.516      0.012       2.865      23.198
C(raion)[T.Великий Новгород]                                        25.8892      5.035      5.142      0.000      16.006      35.772
C(raion)[T.Верхнебуреинский район]                                   9.4892      5.913      1.605      0.109      -2.116      21.095
C(raion)[T.Волжский район]                                          68.9192      5.035     13.688      0.000      59.036      78.802
C(raion)[T.Волотовский район]                                       20.5874      5.183      3.972      0.000      10.414      30.761
C(raion)[T.Вохомский район]                                         75.3158      5.035     14.958      0.000      65.433      85.199
C(raion)[T.Вяземский район]                                         -4.4086      5.035     -0.876      0.382     -14.292       5.475
C(raion)[T.Галичский район]                                         74.1758      5.035     14.732      0.000      64.293      84.059
C(raion)[T.Горномарийский район]                                    81.1869      5.035     16.124      0.000      71.304      91.070
C(raion)[T.Город Волжск]                                            19.3881      5.035      3.851      0.000       9.505      29.271
C(raion)[T.Город Горно-Алтайск]                                     15.6792      5.035      3.114      0.002       5.796      25.562
C(raion)[T.Город Козьмодемьянск]                                    55.4984      6.354      8.735      0.000      43.027      67.970
C(raion)[T.Город Комсомо

In [18]:
treat_1_roads.head()

,raion,region,bad_roads,year,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
281,Печенгский район,Мурманская область,9.00,2015,0,0,0,0,0,0,0,0,0,0,0,0
282,Печенгский район,Мурманская область,6.26,2016,0,0,0,0,0,0,0,0,0,0,0,0
283,Печенгский район,Мурманская область,7.86,2017,0,0,0,0,0,0,0,0,0,0,0,0
284,Печенгский район,Мурманская область,11.72,2018,0,0,0,0,0,0,0,0,0,0,0,0
285,Печенгский район,Мурманская область,24.80,2019,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
treat_1_schools = full_ind_data[["raion", "region", "bad_schools", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_schools).fit()
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     3.212
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.91e-20
Time:                        23:03:26   Log-Likelihood:                -2419.5
No. Observations:                 638   AIC:                             5093.
Df Residuals:                     511   BIC:                             5659.
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               52.5186      8.654      6.069      0.000      35.518      69.520
C(raion)[T.Анучинский район]           -51.7214     10.061     -5.141      0.000     -71.487     -31.956
C(raion)[T.Арсеньевский]               -51.7214     10.061     -5.141      0.000     -71.487     -31.956
C(raion)[T.Артемовский]                -55.5796      9.829     -5.654      0.000     -74.891     -36.268
C(raion)[T.Аяно-Майский район]         -25.4836     14.716     -1.732      0.084     -54.395       3.428
C(raion)[T.Батецкий район]             -44.9998      9.822     -4.582      0.000     -64.296     -25.703
C(raion)[T.Бикинский район]            -41.7138     12.035     -3.466      0.001     -65.358     -18.070
C(raion)[T.Боровичский район]          -44.1971     12.319     -3.588      0.000     -68.398     -19.996
C(raion)[T.Валдайский район]           -51.6665      9.822     -5.260      0.000     -70.963     -32.370
C(raion)[T.Ванинский район]            -45.2441     14.797     -3.058      0.002     -74.315     -16.173
C(raion)[T.Великий Новгород]           -47.2182      9.418     -5.014      0.000     -65.721     -28.716
C(raion)[T.Владивостокский]            -51.7214     10.061     -5.141      0.000     -71.487     -31.956
C(raion)[T.Волжский район]             -52.1960      9.418     -5.542      0.000     -70.698     -33.694
C(raion)[T.Волотовский район]          -34.9998      9.822     -3.563      0.000     -54.296     -15.703
C(raion)[T.Горномарийский район]       -50.1671      9.648     -5.200      0.000     -69.121     -31.213
C(raion)[T.Город Волжск]               -51.9800      9.519     -5.461      0.000     -70.682     -33.278
C(raion)[T.Город Горно-Алтайск]        -50.6960      9.418     -5.383      0.000     -69.198     -32.194
C(raion)[T.Город Йошкар-Ола]           -49.0425      9.519     -5.152      0.000     -67.744     -30.341
C(raion)[T.Город Козьмодемьянск]       -51.7528      9.648     -5.364      0.000     -70.707     -32.799
C(raion)[T.Город Новомосковск]         -55.3173      9.830     -5.627      0.000     -74.630     -36.004
C(raion)[T.Город Тула]                 -50.2867     10.062     -4.998      0.000     -70.055     -30.518
C(raion)[T.Дальнегорский]              -49.0414     10.061     -4.875      0.000     -68.807     -29.276
C(raion)[T.Дальнереченский]            -48.8614     10.061     -4.857      0.000     -68.627     -29.096
C(raion)[T.Демянский район]            -39.1665      9.822     -3.988      0.000     -58.463     -19.870
C(raion)[T.ЗАТО Фокино]                -52.8691     11.014     -4.800      0.000     -74.508     -31.230
C(raion)[T.Заокский район]             -10.9124     12.137     -0.899      0.369     -34.757      12.932
C(raion)[T.Звениговский район]        

In [20]:
treat_1_sewerage =  full_ind_data[["raion", "region", "year", "sewage_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_sewerage).fit()
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     5.457
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.18e-36
Time:                        23:03:26   Log-Likelihood:                 1043.0
No. Observations:                 491   AIC:                            -1848.
Df Residuals:                     372   BIC:                            -1349.
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0074      0.015      0.476      0.635      -0.023       0.038
C(raion)[T.Анучинский район]                                        -0.0066      0.037     -0.180      0.857      -0.079       0.066
C(raion)[T.Арсеньевский]                                            -0.0088      0.024     -0.363      0.717      -0.057       0.039
C(raion)[T.Артемовский]                                             -0.0055      0.019     -0.280      0.780      -0.044       0.033
C(raion)[T.Бикинский район]                                          0.0019      0.028      0.067      0.947      -0.053       0.057
C(raion)[T.Боровичский район]                                       -0.0216      0.024     -0.887      0.376      -0.069       0.026
C(raion)[T.Буйский район]                                           -0.0056      0.028     -0.200      0.841      -0.061       0.049
C(raion)[T.Валдайский район]                                         0.0046      0.019      0.247      0.805      -0.032       0.041
C(raion)[T.Ванинский район]                                         -0.0062      0.020     -0.320      0.749      -0.045       0.032
C(raion)[T.Великий Новгород]                                        -0.0016      0.020     -0.084      0.933      -0.040       0.037
C(raion)[T.Верхнебуреинский район]                                   0.0074      0.028      0.266      0.790      -0.048       0.062
C(raion)[T.Владивостокский]                                         -0.0055      0.019     -0.294      0.769      -0.042       0.031
C(raion)[T.Вяземский район]                                         -0.0063      0.022     -0.281      0.779      -0.050       0.038
C(raion)[T.Галичский район]                                          0.0369      0.037      1.006      0.315      -0.035       0.109
C(raion)[T.Город Горно-Алтайск]                                      0.0025      0.037      0.068      0.946      -0.070       0.075
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0051      0.020     -0.263      0.793      -0.044       0.033
C(raion)[T.Город Новомосковск]                                      -0.0050      0.028     -0.179      0.858      -0.060       0.050
C(raion)[T.Город Тула]                                              -0.0045      0.022     -0.203      0.839      -0.049       0.039
C(raion)[T.Город Хабаровск]                                          0.0015      0.019      0.083      0.934      -0.035       0.038
C(raion)[T.Дальнегорский]                                           -0.0084      0.020     -0.432      0.666      -0.047       0.030
C(raion)[T.Дальнереченск

In [21]:
treat_1_heat = full_ind_data[["raion", "region", "year", "heat_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_heat).fit() 
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     2.076
Date:                Fri, 12 May 2023   Prob (F-statistic):           0.000269
Time:                        23:03:27   Log-Likelihood:                 627.07
No. Observations:                 260   AIC:                            -1160.
Df Residuals:                     213   BIC:                            -992.8
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0279      0.011      2.433      0.016       0.005       0.050
C(raion)[T.Арсеньевский]             -0.0008      0.013     -0.058      0.953      -0.027       0.026
C(raion)[T.Артемовский]              -0.0086      0.013     -0.638      0.524      -0.035       0.018
C(raion)[T.Владивостокский]           0.0039      0.013      0.289      0.773      -0.023       0.030
C(raion)[T.Дальнегорский]            -0.0068      0.013     -0.508      0.612      -0.033       0.020
C(raion)[T.Дальнереченский]           0.0094      0.013      0.699      0.485      -0.017       0.036
C(raion)[T.ЗАТО Фокино]               0.0549      0.015      3.765      0.000       0.026       0.084
C(raion)[T.Кавалеровский район]       0.0014      0.021      0.065      0.948      -0.040       0.043
C(raion)[T.Кировский район]           0.0094      0.013      0.699      0.486      -0.017       0.036
C(raion)[T.Красноармейский район]     0.0284      0.021      1.341      0.181      -0.013       0.070
C(raion)[T.Лазовский район]           0.0379      0.014      2.758      0.006       0.011       0.065
C(raion)[T.Лесозаводский]             0.0152      0.013      1.129      0.260      -0.011       0.042
C(raion)[T.Михайловский район]        0.0105      0.013      0.785      0.434      -0.016       0.037
C(raion)[T.Надеждинский район]        0.0260      0.013      1.936      0.054      -0.000       0.053
C(raion)[T.Находкинский]              0.0097      0.013      0.724      0.470      -0.017       0.036
C(raion)[T.Октябрьский район]         0.0138      0.014      1.006      0.316      -0.013       0.041
C(raion)[T.Ольгинский район]          0.0053      0.013      0.393      0.695      -0.021       0.032
C(raion)[T.Партизанский]              0.0019      0.013      0.142      0.888      -0.025       0.028
C(raion)[T.Пограничный район]         0.0343      0.015      2.359      0.019       0.006       0.063
C(raion)[T.Пожарский район]          -0.0101      0.013     -0.748      0.455      -0.037       0.016
C(raion)[T.Спасск-Дальний]            0.0023      0.013      0.168      0.867      -0.024       0.029
C(raion)[T.Спасский район]           -0.0019      0.013     -0.138      0.890      -0.028       0.025
C(raion)[T.Тернейский район]         -0.0067      0.014     -0.485      0.628      -0.034       0.020
C(raion)[T.Уссурийский]               0.0177      0.013      1.317      0.189      -0.009       0.044
C(raion)[T.Ханкайский район]       6.047e-05      0.014      0.004      0.996      -0.027       0.027
C(raion)[T.Хасанский район]           0.0113      0.013      0.840      0.402      -0.015       0.038
C(raion)[T.Хорольский район]          0.0178      0.014      1.295      0.197      -0.009       0.045
C(raion)[T.Черниговский

In [22]:
treat_1_water = full_ind_data[["raion", "region", "year", "water_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_water).fit()
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     2.774
Date:                Fri, 12 May 2023   Prob (F-statistic):           7.02e-17
Time:                        23:03:27   Log-Likelihood:                 1547.4
No. Observations:                 756   AIC:                            -2837.
Df Residuals:                     627   BIC:                            -2240.
Df Model:                         128                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0414      0.012      3.450      0.001       0.018       0.065
C(raion)[T.Антроповский район]                                      -0.0311      0.023     -1.354      0.176      -0.076       0.014
C(raion)[T.Анучинский район]                                        -0.0176      0.018     -0.969      0.333      -0.053       0.018
C(raion)[T.Арсеньевский]                                            -0.0311      0.023     -1.354      0.176      -0.076       0.014
C(raion)[T.Артемовский]                                             -0.0161      0.016     -0.994      0.320      -0.048       0.016
C(raion)[T.Батецкий район]                                          -0.0308      0.019     -1.605      0.109      -0.068       0.007
C(raion)[T.Бикинский район]                                         -0.0025      0.021     -0.121      0.904      -0.043       0.038
C(raion)[T.Боровичский район]                                       -0.0297      0.025     -1.184      0.237      -0.079       0.020
C(raion)[T.Буйский район]                                           -0.0286      0.018     -1.578      0.115      -0.064       0.007
C(raion)[T.Валдайский район]                                        -0.0195      0.016     -1.208      0.227      -0.051       0.012
C(raion)[T.Ванинский район]                                         -0.0282      0.016     -1.743      0.082      -0.060       0.004
C(raion)[T.Великий Новгород]                                        -0.0331      0.017     -1.985      0.048      -0.066      -0.000
C(raion)[T.Верхнебуреинский район]                                  -0.0137      0.019     -0.715      0.475      -0.051       0.024
C(raion)[T.Владивостокский]                                         -0.0288      0.016     -1.782      0.075      -0.061       0.003
C(raion)[T.Волотовский район]                                       -0.0083      0.019     -0.434      0.664      -0.046       0.029
C(raion)[T.Вохомский район]                                         -0.0215      0.019     -1.123      0.262      -0.059       0.016
C(raion)[T.Вяземский район]                                         -0.0212      0.017     -1.270      0.205      -0.054       0.012
C(raion)[T.Галичский район]                                         -0.0345      0.019     -1.798      0.073      -0.072       0.003
C(raion)[T.Город Горно-Алтайск]                                     -0.0283      0.016     -1.748      0.081      -0.060       0.003
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0155      0.016     -0.958      0.338      -0.047       0.016
C(raion)[T.Город Хабаров

In [23]:
table = pd.DataFrame(np.array([roads.params[(len(roads.params)-10):(len(roads.params))].values, 
                               schools.params[(len(schools.params)-10):(len(schools.params))].values, 
                               sewerage.params[(len(sewerage.params)-10):(len(sewerage.params))].values,
                               heat.params[(len(heat.params)-10):(len(heat.params))].values,
                               water.params[(len(water.params)-10):(len(water.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Качество дорог", "Состояние дошкольных учреждений", "Объем ремонта канализационной сети", "Объем ремонта тепловой сети", "Объем ремонта водопроводной сети"])

In [24]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная
0,-1.315000,8.087880,5.929116,4.211404,-2.169005,0.657343,0.230652,0.453004,-0.049988,1.802776,Качество дорог
1,-12.291013,-10.916491,-8.264567,-5.130167,-8.565588,0.351294,-9.502765,-6.962367,0.000000,0.000000,Состояние дошкольных учреждений
2,0.014541,0.019790,0.015591,0.032097,0.031334,0.019759,0.010887,0.013724,-0.006566,0.008754,Объем ремонта канализационной сети
3,-0.027134,-0.002340,0.006403,0.002837,-0.004186,0.016120,-0.016953,-0.002834,0.000000,0.000000,Объем ремонта тепловой сети
4,0.031400,-0.000536,0.008436,-0.016589,-0.000556,-0.001282,0.010369,-0.000778,-0.005852,0.000000,Объем ремонта водопроводной сети


In [25]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_50_cor.xlsx")

# Models for financial data

In [26]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [27]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [28]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [29]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [30]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [31]:
frames = [actual, initial, difference]

In [32]:
for j in range(len(frames)):
    #temp_fin_data = pd.DataFrame()
    #for j in mapping.keys():
    #    temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 5))])
    frames[j]["treatment_period"] = [1 if i > 2018 else 0 for i in  frames[j].year]
    frames[j] = pd.merge(frames[j], data_for_models, on=["region", "raion"], how="inner").reset_index(drop=True)

In [33]:
for j in range(len(frames)): 
    for i in range(-8,4):
        if i < 0:
            string = f"minus_{abs(i)}"
        else:
            string = f'plus_{i}'
        frames[j][string] = 0
        for region in mapping.keys():
            frames[j].loc[(frames[j].region == region) & (frames[j].year == mapping[region]+5+i) & (frames[j].treatment_1 == 1), string] = 1

In [34]:
frames[0].loc[frames[0].treatment_1 == 1]

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
90,Костромская область,Межевской район,2013,119.932022,447.681476,13080.116533,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
91,Костромская область,Межевской район,2014,268.793039,305.110363,14721.070617,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
92,Костромская область,Межевской район,2015,366.898199,960.397961,21905.499499,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
93,Костромская область,Межевской район,2016,692.608602,752.513135,12733.019118,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
94,Костромская область,Межевской район,2017,NaN,NaN,NaN,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,Тульская область,Ясногорский район,2018,3154.085535,875.742065,11384.009946,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1336,Тульская область,Ясногорский район,2019,2405.870106,745.600890,11821.274934,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1337,Тульская область,Ясногорский район,2020,NaN,NaN,NaN,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1338,Тульская область,Ясногорский район,2021,NaN,4616.592141,13202.097917,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [35]:
frames[2].year.max()

2022

In [36]:
#frames[0]

In [37]:
actual, initial, difference = frames

In [38]:
actual.head()

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
actual_roads_data = actual[["raion", "region", "roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_roads_data).fit()
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     5.029
Date:                Fri, 12 May 2023   Prob (F-statistic):           4.96e-59
Time:                        23:03:42   Log-Likelihood:                -9487.7
No. Observations:                1147   AIC:                         1.931e+04
Df Residuals:                     978   BIC:                         2.017e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          289.0672    353.276      0.818      0.413    -404.200     982.334
C(raion)[T.Антроповский район]                                     561.6978    572.461      0.981      0.327    -561.695    1685.090
C(raion)[T.Анучинский район]                                       778.3790    516.877      1.506      0.132    -235.936    1792.694
C(raion)[T.Арсеньевский]                                           512.3630    483.208      1.060      0.289    -435.880    1460.606
C(raion)[T.Артемовский]                                            981.1595    483.208      2.031      0.043      32.917    1929.402
C(raion)[T.Аяно-Майский район]                                    2560.3153    483.208      5.299      0.000    1612.072    3508.558
C(raion)[T.Батецкий район]                                        -247.2697    483.208     -0.512      0.609   -1195.513     700.973
C(raion)[T.Бикинский район]                                       -320.8584    483.208     -0.664      0.507   -1269.101     627.384
C(raion)[T.Боровичский район]                                      457.2292    592.797      0.771      0.441    -706.071    1620.530
C(raion)[T.Буйский район]                                          884.3096    572.461      1.545      0.123    -239.083    2007.702
C(raion)[T.Валдайский район]                                       169.6356    483.208      0.351      0.726    -778.607    1117.878
C(raion)[T.Ванинский район]                                        -81.9262    483.208     -0.170      0.865   -1030.169     866.317
C(raion)[T.Великий Новгород]                                      1361.8925    483.208      2.818      0.005     413.650    2310.135
C(raion)[T.Верхнебуреинский район]                                 732.7835    483.208      1.516      0.130    -215.459    1681.026
C(raion)[T.Владивостокский]                                       2130.3599    483.208      4.409      0.000    1182.117    3078.603
C(raion)[T.Волжский район]                                          39.9023    498.232      0.080      0.936    -937.824    1017.629
C(raion)[T.Волотовский район]                                     -155.3147    498.232     -0.312      0.755   -1133.041     822.412
C(raion)[T.Вохомский район]                                       2150.2669    572.465      3.756      0.000    1026.865    3273.669
C(raion)[T.Вяземский район]                                        121.2266    483.208      0.251      0.802    -827.016    1069.469
C(raion)[T.Галичский район]                                        850.8148    540.717      1.573      0.116    -210.285    1911.914
C(raion)[T.Горномарийски

In [40]:
actual_housing_data = actual[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_housing_data).fit()
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     45.04
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:03:43   Log-Likelihood:                -11274.
No. Observations:                1163   AIC:                         2.289e+04
Df Residuals:                     994   BIC:                         2.374e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         2222.8739   1459.681      1.523      0.128    -641.537    5087.284
C(raion)[T.Антроповский район]                                    -375.9042   2370.463     -0.159      0.874   -5027.590    4275.781
C(raion)[T.Анучинский район]                                     -1139.0373   2140.332     -0.532      0.595   -5339.124    3061.050
C(raion)[T.Арсеньевский]                                           799.8042   2000.927      0.400      0.689   -3126.722    4726.330
C(raion)[T.Артемовский]                                            856.0682   2000.927      0.428      0.669   -3070.458    4782.594
C(raion)[T.Аяно-Майский район]                                    5.699e+04   2000.927     28.483      0.000    5.31e+04    6.09e+04
C(raion)[T.Батецкий район]                                        -442.0961   2000.927     -0.221      0.825   -4368.622    3484.430
C(raion)[T.Бикинский район]                                       1084.5129   2000.927      0.542      0.588   -2842.013    5011.039
C(raion)[T.Боровичский район]                                    -1232.9348   2408.189     -0.512      0.609   -5958.652    3492.783
C(raion)[T.Буйский район]                                        -1364.5365   2554.862     -0.534      0.593   -6378.078    3649.006
C(raion)[T.Валдайский район]                                      -750.3068   2000.927     -0.375      0.708   -4676.833    3176.219
C(raion)[T.Ванинский район]                                       9769.9337   2000.927      4.883      0.000    5843.408    1.37e+04
C(raion)[T.Великий Новгород]                                       921.6896   2000.927      0.461      0.645   -3004.837    4848.216
C(raion)[T.Верхнебуреинский район]                                5032.4429   2000.927      2.515      0.012    1105.917    8958.969
C(raion)[T.Владивостокский]                                        752.1343   2000.927      0.376      0.707   -3174.392    4678.660
C(raion)[T.Волжский район]                                        -958.5432   2063.136     -0.465      0.642   -5007.145    3090.059
C(raion)[T.Волотовский район]                                      851.4430   2063.136      0.413      0.680   -3197.159    4900.045
C(raion)[T.Вохомский район]                                        372.0768   2370.470      0.157      0.875   -4279.624    5023.778
C(raion)[T.Вяземский район]                                       2856.1783   2000.927      1.427      0.154   -1070.348    6782.704
C(raion)[T.Галичский район]                                      -1157.4452   2239.039     -0.517      0.605   -5551.231    3236.341
C(raion)[T.Горномарийски

In [41]:
actual_education_data = actual[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_education_data).fit()
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     53.85
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:03:43   Log-Likelihood:                -11048.
No. Observations:                1182   AIC:                         2.243e+04
Df Residuals:                    1013   BIC:                         2.329e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.772e+04   1029.691     17.205      0.000    1.57e+04    1.97e+04
C(raion)[T.Антроповский район]                                   -1883.8090   1673.670     -1.126      0.261   -5168.065    1400.447
C(raion)[T.Анучинский район]                                     -3844.8143   1511.210     -2.544      0.011   -6810.275    -879.354
C(raion)[T.Арсеньевский]                                         -6579.0110   1412.782     -4.657      0.000   -9351.324   -3806.698
C(raion)[T.Артемовский]                                          -6738.1640   1412.782     -4.769      0.000   -9510.477   -3965.851
C(raion)[T.Аяно-Майский район]                                    4.413e+04   1412.782     31.237      0.000    4.14e+04    4.69e+04
C(raion)[T.Батецкий район]                                       -4923.0793   1412.782     -3.485      0.001   -7695.393   -2150.766
C(raion)[T.Бикинский район]                                       -657.5799   1412.782     -0.465      0.642   -3429.893    2114.733
C(raion)[T.Боровичский район]                                    -6523.3834   1700.175     -3.837      0.000   -9859.651   -3187.116
C(raion)[T.Буйский район]                                        -3115.3760   1673.670     -1.861      0.063   -6399.632     168.880
C(raion)[T.Валдайский район]                                     -7065.9048   1412.782     -5.001      0.000   -9838.218   -4293.591
C(raion)[T.Ванинский район]                                        504.7007   1412.782      0.357      0.721   -2267.613    3277.014
C(raion)[T.Великий Новгород]                                     -8048.9226   1412.782     -5.697      0.000   -1.08e+04   -5276.609
C(raion)[T.Верхнебуреинский район]                                6874.1566   1412.782      4.866      0.000    4101.843    9646.470
C(raion)[T.Владивостокский]                                      -8211.9027   1412.782     -5.813      0.000    -1.1e+04   -5439.589
C(raion)[T.Волжский район]                                       -6146.0971   1456.709     -4.219      0.000   -9004.609   -3287.585
C(raion)[T.Волотовский район]                                    -7129.3497   1456.709     -4.894      0.000   -9987.862   -4270.837
C(raion)[T.Вохомский район]                                        373.9231   1673.684      0.223      0.823   -2910.361    3658.207
C(raion)[T.Вяземский район]                                       3552.5245   1412.782      2.515      0.012     780.211    6324.838
C(raion)[T.Галичский район]                                      -3949.2301   1580.888     -2.498      0.013   -7051.421    -847.039
C(raion)[T.Горномарийски

In [42]:
initial_roads_data = initial[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_roads_data).fit()
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     2.348
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.06e-13
Time:                        23:03:44   Log-Likelihood:                -7859.8
No. Observations:                 911   AIC:                         1.601e+04
Df Residuals:                     764   BIC:                         1.672e+04
Df Model:                         146                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           86.6249    549.502      0.158      0.875    -992.087    1165.337
C(raion)[T.Антроповский район]                                     675.1916    737.734      0.915      0.360    -773.035    2123.418
C(raion)[T.Анучинский район]                                       812.0380    842.169      0.964      0.335    -841.201    2465.277
C(raion)[T.Арсеньевский]                                           161.1737    737.734      0.218      0.827   -1287.053    1609.400
C(raion)[T.Артемовский]                                            492.8104    737.734      0.668      0.504    -955.416    1941.037
C(raion)[T.Аяно-Майский район]                                    2700.7786    737.734      3.661      0.000    1252.552    4149.005
C(raion)[T.Батецкий район]                                        -162.5399    764.026     -0.213      0.832   -1662.379    1337.299
C(raion)[T.Бикинский район]                                       -132.7269    737.734     -0.180      0.857   -1580.953    1315.499
C(raion)[T.Боровичский район]                                      791.1857    877.524      0.902      0.368    -931.458    2513.829
C(raion)[T.Буйский район]                                          486.6262    764.026      0.637      0.524   -1013.213    1986.465
C(raion)[T.Валдайский район]                                       186.8944    764.026      0.245      0.807   -1312.944    1686.733
C(raion)[T.Ванинский район]                                        -26.5885    737.734     -0.036      0.971   -1474.815    1421.638
C(raion)[T.Великий Новгород]                                      -214.6817    737.734     -0.291      0.771   -1662.908    1233.545
C(raion)[T.Верхнебуреинский район]                                -215.3413    737.734     -0.292      0.770   -1663.568    1232.885
C(raion)[T.Владивостокский]                                       1271.0419    737.734      1.723      0.085    -177.184    2719.268
C(raion)[T.Волотовский район]                                     -169.0638    763.871     -0.221      0.825   -1668.599    1330.471
C(raion)[T.Вохомский район]                                        734.4254    764.026      0.961      0.337    -765.413    2234.264
C(raion)[T.Вяземский район]                                        109.7405    737.734      0.149      0.882   -1338.486    1557.967
C(raion)[T.Галичский район]                                        743.8359    764.026      0.974      0.331    -756.003    2243.675
C(raion)[T.Город Горно-Алтайск]                                   1261.8088    905.093      1.394      0.164    -514.956    3038.574
C(raion)[T.Город Комсомо

In [43]:
initial_housing_data = initial[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_housing_data).fit()
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     15.87
Date:                Fri, 12 May 2023   Prob (F-statistic):          2.36e-158
Time:                        23:03:44   Log-Likelihood:                -9364.2
No. Observations:                 929   AIC:                         1.902e+04
Df Residuals:                     781   BIC:                         1.974e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          796.2269   2319.400      0.343      0.731   -3756.769    5349.223
C(raion)[T.Антроповский район]                                   -1046.5300   3862.154     -0.271      0.786   -8627.962    6534.902
C(raion)[T.Анучинский район]                                      -400.8795   3593.590     -0.112      0.911   -7455.118    6653.359
C(raion)[T.Арсеньевский]                                           928.4250   3148.239      0.295      0.768   -5251.588    7108.438
C(raion)[T.Артемовский]                                            184.4478   3148.239      0.059      0.953   -5995.565    6364.460
C(raion)[T.Аяно-Майский район]                                    4.955e+04   3148.239     15.740      0.000    4.34e+04    5.57e+04
C(raion)[T.Батецкий район]                                        -918.2800   3260.064     -0.282      0.778   -7317.806    5481.246
C(raion)[T.Бикинский район]                                       1277.5350   3148.239      0.406      0.685   -4902.478    7457.548
C(raion)[T.Боровичский район]                                     -884.7778   3712.979     -0.238      0.812   -8173.378    6403.823
C(raion)[T.Буйский район]                                         -945.1988   3260.064     -0.290      0.772   -7344.725    5454.327
C(raion)[T.Валдайский район]                                      -912.5383   3403.192     -0.268      0.789   -7593.025    5767.949
C(raion)[T.Ванинский район]                                       9064.7576   3148.239      2.879      0.004    2884.745    1.52e+04
C(raion)[T.Великий Новгород]                                      -958.7096   3148.239     -0.305      0.761   -7138.722    5221.303
C(raion)[T.Верхнебуреинский район]                                -957.1933   3148.239     -0.304      0.761   -7137.206    5222.819
C(raion)[T.Владивостокский]                                        590.1534   3148.239      0.187      0.851   -5589.859    6770.166
C(raion)[T.Волотовский район]                                    -1001.7330   3259.763     -0.307      0.759   -7400.667    5397.201
C(raion)[T.Вохомский район]                                       -743.2496   3260.064     -0.228      0.820   -7142.775    5656.276
C(raion)[T.Вяземский район]                                       2499.8620   3148.239      0.794      0.427   -3680.151    8679.875
C(raion)[T.Галичский район]                                       -784.6508   3260.064     -0.241      0.810   -7184.177    5614.875
C(raion)[T.Город Горно-Алтайск]                                   1172.8660   3862.154      0.304      0.761   -6408.566    8754.298
C(raion)[T.Город Комсомо

In [44]:
initial_education_data = initial[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_education_data).fit()
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     104.1
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:03:45   Log-Likelihood:                -8532.1
No. Observations:                 953   AIC:                         1.736e+04
Df Residuals:                     805   BIC:                         1.808e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.575e+04    749.197     21.021      0.000    1.43e+04    1.72e+04
C(raion)[T.Антроповский район]                                   -6153.2268   1017.572     -6.047      0.000   -8150.635   -4155.819
C(raion)[T.Анучинский район]                                     -2257.3575   1161.489     -1.944      0.052   -4537.262      22.547
C(raion)[T.Арсеньевский]                                         -5571.2562   1017.572     -5.475      0.000   -7568.664   -3573.848
C(raion)[T.Артемовский]                                          -6552.1787   1017.572     -6.439      0.000   -8549.587   -4554.771
C(raion)[T.Аяно-Майский район]                                    3.437e+04   1017.572     33.778      0.000    3.24e+04    3.64e+04
C(raion)[T.Батецкий район]                                       -4363.0708   1053.709     -4.141      0.000   -6431.413   -2294.729
C(raion)[T.Бикинский район]                                      -1732.6383   1017.572     -1.703      0.089   -3730.046     264.770
C(raion)[T.Боровичский район]                                    -5785.2165   1198.697     -4.826      0.000   -8138.157   -3432.276
C(raion)[T.Буйский район]                                        -4889.7591   1053.709     -4.641      0.000   -6958.101   -2821.417
C(raion)[T.Валдайский район]                                     -6646.9296   1053.709     -6.308      0.000   -8715.272   -4578.587
C(raion)[T.Ванинский район]                                       2182.6928   1017.572      2.145      0.032     185.285    4180.101
C(raion)[T.Великий Новгород]                                     -1.514e+04   1017.572    -14.877      0.000   -1.71e+04   -1.31e+04
C(raion)[T.Верхнебуреинский район]                               -1.513e+04   1017.572    -14.864      0.000   -1.71e+04   -1.31e+04
C(raion)[T.Владивостокский]                                      -7449.3004   1017.572     -7.321      0.000   -9446.708   -5451.892
C(raion)[T.Волотовский район]                                    -1.501e+04   1053.620    -14.242      0.000   -1.71e+04   -1.29e+04
C(raion)[T.Вохомский район]                                      -3164.8992   1053.709     -3.004      0.003   -5233.241   -1096.557
C(raion)[T.Вяземский район]                                       4639.9189   1017.572      4.560      0.000    2642.511    6637.327
C(raion)[T.Галичский район]                                      -5045.3199   1053.709     -4.788      0.000   -7113.662   -2976.978
C(raion)[T.Город Горно-Алтайск]                                  -2583.4637   1248.250     -2.070      0.039   -5033.673    -133.254
C(raion)[T.Город Комсомо

In [45]:
difference_roads_data = difference[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_roads_data).fit()
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     2.615
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.95e-16
Time:                        23:03:45   Log-Likelihood:                -6454.4
No. Observations:                 798   AIC:                         1.320e+04
Df Residuals:                     652   BIC:                         1.388e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          100.7559    325.297      0.310      0.757    -538.001     739.513
C(raion)[T.Антроповский район]                                     212.8504    534.766      0.398      0.691    -837.220    1262.921
C(raion)[T.Анучинский район]                                       417.2913    497.569      0.839      0.402    -559.739    1394.322
C(raion)[T.Арсеньевский]                                           423.2823    435.791      0.971      0.332    -432.441    1279.006
C(raion)[T.Артемовский]                                            610.1817    435.791      1.400      0.162    -245.542    1465.905
C(raion)[T.Аяно-Майский район]                                       6.9539    435.791      0.016      0.987    -848.770     862.677
C(raion)[T.Батецкий район]                                        -155.1201    451.332     -0.344      0.731   -1041.361     731.120
C(raion)[T.Бикинский район]                                       -142.2890    435.791     -0.327      0.744    -998.013     713.435
C(raion)[T.Боровичский район]                                      139.2008    542.050      0.257      0.797    -925.174    1203.575
C(raion)[T.Буйский район]                                          108.0541    591.496      0.183      0.855   -1053.414    1269.522
C(raion)[T.Валдайский район]                                      -145.2304    451.332     -0.322      0.748   -1031.471     741.010
C(raion)[T.Ванинский район]                                         26.8521    435.791      0.062      0.951    -828.871     882.576
C(raion)[T.Великий Новгород]                                      1628.1248    435.791      3.736      0.000     772.401    2483.848
C(raion)[T.Верхнебуреинский район]                                1079.4391    435.791      2.477      0.014     223.715    1935.163
C(raion)[T.Владивостокский]                                        957.9168    435.791      2.198      0.028     102.193    1813.640
C(raion)[T.Волотовский район]                                       -2.2081    451.257     -0.005      0.996    -888.300     883.884
C(raion)[T.Вохомский район]                                       2372.1774    591.512      4.010      0.000    1210.678    3533.676
C(raion)[T.Вяземский район]                                        -52.6785    435.791     -0.121      0.904    -908.402     803.045
C(raion)[T.Галичский район]                                        592.5125    534.725      1.108      0.268    -457.478    1642.503
C(raion)[T.Город Горно-Алтайск]                                   1909.5449    534.772      3.571      0.000     859.461    2959.629
C(raion)[T.Город Комсомо

In [46]:
difference_housing_data = difference[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_housing_data).fit()
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     2.505
Date:                Fri, 12 May 2023   Prob (F-statistic):           3.62e-15
Time:                        23:03:46   Log-Likelihood:                -7953.9
No. Observations:                 816   AIC:                         1.620e+04
Df Residuals:                     670   BIC:                         1.689e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1395.6443   1687.664      0.827      0.409   -1918.103    4709.392
C(raion)[T.Антроповский район]                                     727.2318   4867.353      0.149      0.881   -8829.868    1.03e+04
C(raion)[T.Анучинский район]                                      -595.4406   2608.272     -0.228      0.819   -5716.812    4525.931
C(raion)[T.Арсеньевский]                                          -158.4673   2284.606     -0.069      0.945   -4644.317    4327.383
C(raion)[T.Артемовский]                                            563.8791   2284.606      0.247      0.805   -3921.971    5049.729
C(raion)[T.Аяно-Майский район]                                    8276.4726   2284.606      3.623      0.000    3790.623    1.28e+04
C(raion)[T.Батецкий район]                                        -166.6251   2365.821     -0.070      0.944   -4811.941    4478.691
C(raion)[T.Бикинский район]                                       -240.9125   2284.606     -0.105      0.916   -4726.762    4244.937
C(raion)[T.Боровичский район]                                    -1082.4682   2794.667     -0.387      0.699   -6569.827    4404.891
C(raion)[T.Буйский район]                                         -242.9605   3623.365     -0.067      0.947   -7357.477    6871.556
C(raion)[T.Валдайский район]                                      -611.5661   2469.775     -0.248      0.805   -5460.996    4237.864
C(raion)[T.Ванинский район]                                        373.6769   2284.606      0.164      0.870   -4112.173    4859.527
C(raion)[T.Великий Новгород]                                      1823.6450   2284.606      0.798      0.425   -2662.205    6309.495
C(raion)[T.Верхнебуреинский район]                                5891.3618   2284.606      2.579      0.010    1405.512    1.04e+04
C(raion)[T.Владивостокский]                                        158.7834   2284.606      0.070      0.945   -4327.066    4644.633
C(raion)[T.Волотовский район]                                     1915.2356   2365.671      0.810      0.418   -2729.786    6560.257
C(raion)[T.Вохомский район]                                       2143.7076   3100.791      0.691      0.490   -3944.729    8232.144
C(raion)[T.Вяземский район]                                        203.1508   2284.606      0.089      0.929   -4282.699    4689.001
C(raion)[T.Галичский район]                                         83.8679   2803.015      0.030      0.976   -5419.884    5587.620
C(raion)[T.Город Горно-Алтайск]                                    552.0462   2803.274      0.197      0.844   -4952.214    6056.306
C(raion)[T.Город Комсомо

In [47]:
difference_education_data = difference[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_education_data).fit()
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     7.455
Date:                Fri, 12 May 2023   Prob (F-statistic):           6.52e-76
Time:                        23:03:47   Log-Likelihood:                -7770.7
No. Observations:                 834   AIC:                         1.584e+04
Df Residuals:                     686   BIC:                         1.654e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1117.8752   1095.242      1.021      0.308   -1032.553    3268.304
C(raion)[T.Антроповский район]                                    4992.7788   1821.621      2.741      0.006    1416.157    8569.401
C(raion)[T.Анучинский район]                                     -1483.5836   1694.931     -0.875      0.382   -4811.459    1844.292
C(raion)[T.Арсеньевский]                                         -1085.3737   1484.650     -0.731      0.465   -4000.376    1829.629
C(raion)[T.Артемовский]                                           -105.1362   1484.650     -0.071      0.944   -3020.139    2809.866
C(raion)[T.Аяно-Майский район]                                    8617.7915   1484.650      5.805      0.000    5702.789    1.15e+04
C(raion)[T.Батецкий район]                                        -129.9083   1537.405     -0.084      0.933   -3148.493    2888.676
C(raion)[T.Бикинский район]                                        492.5956   1484.650      0.332      0.740   -2422.407    3407.598
C(raion)[T.Боровичский район]                                    -1112.3222   1815.958     -0.613      0.540   -4677.824    2453.180
C(raion)[T.Буйский район]                                         4414.3684   2014.892      2.191      0.029     458.272    8370.465
C(raion)[T.Валдайский район]                                      -189.4856   1537.405     -0.123      0.902   -3208.070    2829.099
C(raion)[T.Ванинский район]                                      -2133.6686   1484.650     -1.437      0.151   -5048.671     781.334
C(raion)[T.Великий Новгород]                                      7110.4670   1484.650      4.789      0.000    4195.464       1e+04
C(raion)[T.Верхнебуреинский район]                                2.198e+04   1484.650     14.805      0.000    1.91e+04    2.49e+04
C(raion)[T.Владивостокский]                                       -766.9939   1484.650     -0.517      0.606   -3681.997    2148.009
C(raion)[T.Волотовский район]                                     7610.0500   1537.326      4.950      0.000    4591.620    1.06e+04
C(raion)[T.Вохомский район]                                       2916.6714   2014.951      1.448      0.148   -1039.539    6872.882
C(raion)[T.Вяземский район]                                      -1247.9966   1484.650     -0.841      0.401   -4162.999    1667.006
C(raion)[T.Галичский район]                                       2885.6309   1821.484      1.584      0.114    -690.723    6461.984
C(raion)[T.Город Горно-Алтайск]                                   1405.2732   1821.636      0.771      0.441   -2171.378    4981.925
C(raion)[T.Город Комсомо

In [48]:
table = pd.DataFrame(np.array([actual_roads.params[(len(actual_roads.params)-10):(len(actual_roads.params))].values, 
                               actual_housing.params[(len(actual_housing.params)-10):(len(actual_housing.params))].values, 
                               actual_education.params[(len(actual_education.params)-10):(len(actual_education.params))].values,
                               initial_roads.params[(len(initial_roads.params)-10):(len(initial_roads.params))].values, 
                               initial_housing.params[(len(initial_housing.params)-10):(len(initial_housing.params))].values, 
                               initial_education.params[(len(initial_education.params)-10):(len(initial_education.params))].values,
                               difference_roads.params[(len(difference_roads.params)-10):(len(difference_roads.params))].values, 
                               difference_housing.params[(len(difference_housing.params)-10):(len(difference_housing.params))].values, 
                               difference_education.params[(len(difference_education.params)-10):(len(difference_education.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование"])
table["Тип расходов"] = pd.Series(["Фактические", "Фактические", "Фактические", "Запланированные", "Запланированные", "Запланированные", "Фактические - запланированные", "Фактические - запланированные", "Фактические - запланированные"])

In [49]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная,Тип расходов
0,-4.505295e+02,-409.876125,-447.900844,-750.496784,-377.768668,-519.019676,-173.173761,-449.242750,-1006.810216,-758.057274,Дорожное хозяйство,Фактические
1,1.303577e+03,818.673134,652.435030,923.682961,880.020638,846.755732,592.192364,681.044567,1650.064212,1510.390906,ЖКХ,Фактические
2,-6.204006e+03,-2832.955205,-2018.808317,1284.754138,-1223.348210,11.200344,-67.595877,1434.895703,-1105.699924,2289.242976,Образование,Фактические
3,-2.274760e-13,-65.633850,-132.979470,-841.480543,-709.085654,-882.721817,-908.198915,-893.107415,-333.231614,-1231.891099,Дорожное хозяйство,Запланированные
4,-2.660354e-12,251.624378,663.489899,315.827176,477.204402,616.400372,-236.846604,-235.641899,-15.135544,406.065906,ЖКХ,Запланированные
5,-3.750865e-13,-575.814092,-1843.472936,-2219.758228,-1769.220013,-1228.468293,464.028269,-1898.814118,-1860.399892,-298.748302,Образование,Запланированные
6,1.625060e-13,988.163879,-700.803341,-424.496820,-477.551222,-383.443783,42.302302,-451.066853,-988.419384,-174.458051,Дорожное хозяйство,Фактические - запланированные
7,-1.625463e-12,426.462247,799.600204,811.734540,850.344151,689.553296,811.653773,727.411406,1012.970745,1550.162550,ЖКХ,Фактические - запланированные
8,1.309521e-12,-779.093317,-1196.404870,2937.144957,296.266349,1150.320992,-1512.270923,3013.952354,-207.230216,2134.992672,Образование,Фактические - запланированные


In [50]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_fin_50_cor.xlsx")